In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

df = pd.read_excel('/Users/liamzeeum/Github/group4-project/Cleaned_Data.xlsx')
df.head()

,Watershed,Latitude,Longitude,StartDate,StartTime,DWM_Name,DWM_Units,ResVal,nResult
0,South Coastal,42.160283,-70.788634,6/25/2019,1:10:00 PM,Ammonia-N,mg/L,<0.20,-0.200
1,South Coastal,42.160283,-70.788634,6/25/2019,1:10:00 PM,Nitrate/Nitrite-N,mg/L,<0.20,-0.200
2,South Coastal,42.160283,-70.788634,6/25/2019,1:10:00 PM,Total Nitrogen,mg/L,0.77,0.770
3,South Coastal,42.160283,-70.788634,6/25/2019,1:10:00 PM,Total Phosphorus,mg/L,0.056,0.056
4,South Coastal,42.187649,-70.768508,6/25/2019,1:50:00 PM,Ammonia-N,mg/L,<0.20,-0.200


In [45]:
df_copy= df.copy()

df_copy['StartDate'] = pd.to_datetime(df_copy['StartDate'], format='%m/%d/%Y')

I'm just adding a dummy variable that = 1 if the sample was taken after 2013 in the code below - Liam

In [46]:
for i, row in df_copy.iterrows():
    if row['StartDate'].year >= 2013:
        df_copy.loc[i, 'dummy_variable'] = 1
    else:
        df_copy.loc[i, 'dummy_variable'] = 0

Here I'm copying in my Haversine formula addition to the dataframe so we can run distance regressions in here - Liam

In [60]:
from math import radians, cos, sin, asin, sqrt

urbanfarm_df = pd.read_excel('/Users/liamzeeum/Github/group4-project/Urban Farm Location Data.xlsx')

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    distance_lon = lon2-lon1
    distance_lat = lat2-lat1
    a = sin(distance_lat/2)**2 + cos(lat1) * cos(lat2) * sin(distance_lon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6367 * c
    return km

distances = []
for i, sample in df_copy.iterrows():
    min_distance = np.inf
    for j, farm in urbanfarm_df.iterrows():
        distance = haversine(sample['Latitude'], sample['Longitude'], farm['Latitude'], farm['Longitude'])
        if distance < min_distance:
            min_distance=distance
    distances.append(min_distance)

df_copy['closest_farm_distance_km'] = distances

df_copy.head()

,Watershed,Latitude,Longitude,StartDate,StartTime,DWM_Name,DWM_Units,ResVal,nResult,dummy_variable,closest_farm_distance_km
0,South Coastal,42.160283,-70.788634,2019-06-25,1:10:00 PM,Ammonia-N,mg/L,<0.20,-0.200,1.0,27.900587
1,South Coastal,42.160283,-70.788634,2019-06-25,1:10:00 PM,Nitrate/Nitrite-N,mg/L,<0.20,-0.200,1.0,27.900587
2,South Coastal,42.160283,-70.788634,2019-06-25,1:10:00 PM,Total Nitrogen,mg/L,0.77,0.770,1.0,27.900587
3,South Coastal,42.160283,-70.788634,2019-06-25,1:10:00 PM,Total Phosphorus,mg/L,0.056,0.056,1.0,27.900587
4,South Coastal,42.187649,-70.768508,2019-06-25,1:50:00 PM,Ammonia-N,mg/L,<0.20,-0.200,1.0,28.037568


**Nitrogen:**

I used the nitrogen subsets made by Liam in "Revisualization" and ran them as a two-sample t-test. With a pval of 0.005 that's smaller than any reasonable signifcance level, it seems that nitrogen levels pre-2013 are significantly greater than nitrogen levels post_2013. However, this doesn't account for regional differences or distance from farms. 

In [62]:
nitrogen_pre2013_df = df_copy.loc[(df_copy['StartDate'].dt.year<2013) & (df_copy['DWM_Name'] == 'Total Nitrogen')]
nitrogen_post2013_df = df_copy.loc[(df_copy['StartDate'].dt.year>=2013) & (df_copy['DWM_Name'] == 'Total Nitrogen')]

stats.ttest_ind(nitrogen_pre2013_df['nResult'], nitrogen_post2013_df['nResult'], alternative='two-sided')

Ttest_indResult(statistic=2.7338781929763725, pvalue=0.0064419014317596964)

In [63]:
nitrogen_post2013_df['nResult'].mean()

0.848504672897196

nitrogen_pre2013.mean = 1.1208964143426299
nitrogen_post2013.mean = 0.848504672897196

In [64]:
import statsmodels.formula.api as smf
nitrogen_df = df_copy.loc[(df_copy['DWM_Name'] == 'Total Nitrogen')]
nitrogen_ols = smf.ols(formula='nResult ~ closest distance dummy_variable', data = nitrogen_df).fit()
nitrogen_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                nResult   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     7.474
Date:                Tue, 02 May 2023   Prob (F-statistic):            0.00644
Time:                        17:39:43   Log-Likelihood:                -822.68
No. Observations:                 609   AIC:                             1649.
Df Residuals:                     607   BIC:                             1658.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.1209      0.042     26.839      0.000       1.039       1.203
dummy_variable    -0.2724      0.100     -2.734      0.006      -0.468      -0.077
==============================================================================
Omnibus:                      575.958   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19607.639
Skew:                           4.242   Prob(JB):                         0.00
Kurtosis:                      29.472   Cond. No.                         2.72
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Next, I ran t-tests for each individual watershed to account for regional differences. Referring back to the map created in check-in 4, there are some watersheds located further away from Boston, while some pass through the heart of Boston, so I would expect to see some differences in significance of pre- and post-2013 data differences.  

In [65]:
GPreSouth = nitrogen_pre2013_df.loc[nitrogen_pre2013_df['Watershed'] == 'South Coastal']
GPreshaw = nitrogen_pre2013_df.loc[nitrogen_pre2013_df['Watershed'] == 'Shawsheen']
GPrenep = nitrogen_pre2013_df.loc[nitrogen_pre2013_df['Watershed'] == 'Neponset']
GPrecharles = nitrogen_pre2013_df.loc[nitrogen_pre2013_df['Watershed'] == 'Charles']

GPostSouth = nitrogen_post2013_df.loc[nitrogen_post2013_df['Watershed'] == 'South Coastal']
GPostshaw = nitrogen_post2013_df.loc[nitrogen_post2013_df['Watershed'] == 'Shawsheen']
GPostnep = nitrogen_post2013_df.loc[nitrogen_post2013_df['Watershed'] == 'Neponset']
GPostcharles = nitrogen_post2013_df.loc[nitrogen_post2013_df['Watershed'] == 'Charles']

In [67]:
NitrogenAllSouthCoastal = df_copy.loc[(df_copy['DWM_Name'] == 'Total Nitrogen') & (df_copy['Watershed'] == 'South Coastal')]
NitrogenAllShaw = df_copy.loc[(df_copy['DWM_Name'] == 'Total Nitrogen') & (df_copy['Watershed'] == 'Shawsheen')]
NitrogenAllNep = df_copy.loc[(df_copy['DWM_Name'] == 'Total Nitrogen') & (df_copy['Watershed'] == 'Neponset')]
NitrogenAllCharles = df_copy.loc[(df_copy['DWM_Name'] == 'Total Nitrogen') & (df_copy['Watershed'] == 'Charles')]

South Coastal:

In [52]:
stats.ttest_ind(GPreSouth['nResult'], GPostSouth['nResult'], alternative='two-sided')

Ttest_indResult(statistic=2.2223825974718316, pvalue=0.027418631301931822)

In [ ]:
nitrogen_south_coastal_ols = smf.ols(formula='nResult ~ dummy_variable', data =NitrogenAllSouthCoastal).fit()
nitrogen_south_coastal_ols.summary()

In [75]:
nitrogen_south_coastal_distance_ols = smf.ols(formula='nResult ~ closest_farm_distance_km + closest_farm_distance_km:dummy_variable', data =NitrogenAllSouthCoastal).fit()
nitrogen_south_coastal_distance_ols.summary()


43.246770313169144

Shawsheen:

In [ ]:
nitrogen_shawsheen_ols = smf.ols(formula='nResult ~ dummy_variable', data =NitrogenAllShaw).fit()
nitrogen_shawsheen_ols.summary()

In [55]:
stats.ttest_ind(GPreshaw['nResult'], GPostshaw['nResult'], alternative='two-sided')

Ttest_indResult(statistic=1.6796379031154025, pvalue=0.09653510203234185)

Neponset: 

In [56]:
stats.ttest_ind(GPrenep['nResult'], GPostnep['nResult'], alternative='two-sided')

Ttest_indResult(statistic=2.7812732926419597, pvalue=0.006390687830296602)

In [ ]:
nitrogen_neponset_ols = smf.ols(formula='nResult ~ dummy_variable', data =NitrogenAllNep).fit()
nitrogen_neponset_ols.summary()

In [76]:
nitrogen_neponset_distance_ols = smf.ols(formula='nResult ~ closest_farm_distance_km + closest_farm_distance_km:dummy_variable', data =NitrogenAllNep).fit()
nitrogen_neponset_distance_ols.summary()
NitrogenAllNep['closest_farm_distance_km'].mean()


7.68816849433671

Charles:

In [58]:
stats.ttest_ind(GPrecharles['nResult'], GPostcharles['nResult'], alternative='two-sided')

Ttest_indResult(statistic=0.5252952662933288, pvalue=0.5999294962869886)

In [ ]:
nitrogen_charles_ols = smf.ols(formula='nResult ~ dummy_variable', data =NitrogenAllCharles).fit()
nitrogen_charles_ols.summary()

In [ ]:
nitrogen_charles_distance_ols = smf.ols(formula='nResult ~ dummy_variable', data =NitrogenAllCharles).fit()

I ran dummy variable regressions on nitrogen levels in each water shed. The Coefficients reflect the difference in mean nitrogen concentration in each watershed pre-2013 to post-2013. 

Charles: -.1025, not significant 
Neponset: -0.5059, significant 
Shawsheen: -0.1608, not significant 
South Coastal: -0.4191, significant 

Next, I ran a multiple regression on the watersheds that had significant coefficients. I regressed nutrient densities on our distance variable and an interaction term between distance and the 2013 dumy variable. The coefficient on the interaction term will tell us how the effect of distance from farms on nutrient densities changed after 2013.

Results 

Neponset -> Coeffcient on distance = -0.0236, significant at .049 t-value, coefficient on interactoin term -> -0.1030, significant at the .002 t-value. Total model r-squared is .109, mean distance is 7.68816849433671

South Coastal -> Coeffcient on distance = -0.0340 , significant at .049 t-value, coefficient on interactoin term -> -0.0054, not significant Total model r-squared is .178, mean distance = 43.246770313169144

**Ammonia:**

I did the same for ammonia, which shows a pval of 0.1, demonstrating that ammonia levels pre-2013 were not necessarily greater than post-2013. 

In [10]:
ammonia_pre2013_df = df_copy.loc[(df_copy['StartDate'].dt.year<2013) & (df_copy['DWM_Name'] == 'Ammonia-N')]
ammonia_post2013_df = df_copy.loc[(df_copy['StartDate'].dt.year>=2013) & (df_copy['DWM_Name'] == 'Ammonia-N')]

stats.ttest_ind(ammonia_pre2013_df['nResult'], ammonia_post2013_df['nResult'], alternative='two-sided')

Ttest_indResult(statistic=1.2776985236750187, pvalue=0.20193250688987674)

I then ran t-tests by region for Ammonia too.

In [11]:
APreSouth = ammonia_pre2013_df.loc[ammonia_pre2013_df['Watershed'] == 'South Coastal']
APreshaw = ammonia_pre2013_df.loc[ammonia_pre2013_df['Watershed'] == 'Shawsheen']
APrenep = ammonia_pre2013_df.loc[ammonia_pre2013_df['Watershed'] == 'Neponset']
APrecharles = ammonia_pre2013_df.loc[ammonia_pre2013_df['Watershed'] == 'Charles']

APostSouth = ammonia_post2013_df.loc[ammonia_post2013_df['Watershed'] == 'South Coastal']
APostshaw = ammonia_post2013_df.loc[ammonia_post2013_df['Watershed'] == 'Shawsheen']
APostnep = ammonia_post2013_df.loc[ammonia_post2013_df['Watershed'] == 'Neponset']
APostcharles = ammonia_post2013_df.loc[ammonia_post2013_df['Watershed'] == 'Charles']

South Coastal:

In [12]:
stats.ttest_ind(APreSouth['nResult'], APostSouth['nResult'], alternative='two-sided')

Ttest_indResult(statistic=3.5767445550824473, pvalue=0.0005239370574844963)

Shawsheen:

In [13]:
stats.ttest_ind(APreshaw['nResult'], APostshaw['nResult'], alternative='two-sided')

Ttest_indResult(statistic=2.284088498789362, pvalue=0.024718676015365024)

Neponset:

In [14]:
stats.ttest_ind(APrenep['nResult'], APostnep['nResult'], alternative='two-sided')

Ttest_indResult(statistic=1.7835758368450834, pvalue=0.07738164630316936)

Charles:

In [15]:
stats.ttest_ind(APrecharles['nResult'], APostcharles['nResult'], alternative='two-sided')

Ttest_indResult(statistic=-1.7345167742714582, pvalue=0.08432225605159661)

Interestingly, when run individually, South Coastal and Shawsheen show significant difference between the pre- and post-2013, while Neponset and Charles show no significant difference like our original results.

**Phosphorus:**

Phosphorus is more similar to Nitrogen, showing a pval of 0.003 and demonstrating significance.

In [16]:
phosphorus_pre2013_df = df_copy.loc[(df_copy['StartDate'].dt.year<2013) & (df_copy['DWM_Name'] == 'Total Phosphorus')]
phosphorus_post2013_df = df_copy.loc[(df_copy['StartDate'].dt.year>=2013) & (df_copy['DWM_Name'] == 'Total Phosphorus')]

stats.ttest_ind(phosphorus_pre2013_df['nResult'], phosphorus_post2013_df['nResult'], alternative='two-sided')

Ttest_indResult(statistic=2.733644704011147, pvalue=0.006445810386973598)

In [94]:
phosphorus_df = df_copy.loc[(df_copy['DWM_Name'] == 'Total Phosphorus')]
phosphorus_df_ols = smf.ols(formula= 'nResult ~ dummy_variable', data=phosphorus_df).fit()
phosphorus_df_ols.summary()
print(phosphorus_pre2013_df['nResult'].mean())
print(phosphorus_post2013_df['nResult'].mean())


0.05725148514851489
0.03869811320754717


Once again, I run t-tests for each individual watershed.

In [17]:
PPreSouth = phosphorus_pre2013_df.loc[phosphorus_pre2013_df['Watershed'] == 'South Coastal']
PPreshaw = phosphorus_pre2013_df.loc[phosphorus_pre2013_df['Watershed'] == 'Shawsheen']
PPrenep = phosphorus_pre2013_df.loc[phosphorus_pre2013_df['Watershed'] == 'Neponset']
PPrecharles = phosphorus_pre2013_df.loc[phosphorus_pre2013_df['Watershed'] == 'Charles']

PPostSouth = phosphorus_post2013_df.loc[phosphorus_post2013_df['Watershed'] == 'South Coastal']
PPostshaw = phosphorus_post2013_df.loc[phosphorus_post2013_df['Watershed'] == 'Shawsheen']
PPostnep = phosphorus_post2013_df.loc[phosphorus_post2013_df['Watershed'] == 'Neponset']
PPostcharles = phosphorus_post2013_df.loc[phosphorus_post2013_df['Watershed'] == 'Charles']

South Coastal:

In [18]:
stats.ttest_ind(PPreSouth['nResult'], PPostSouth['nResult'], alternative='two-sided')

Ttest_indResult(statistic=2.7662846457675205, pvalue=0.006248491333998326)

Shawsheen:

In [19]:
stats.ttest_ind(PPreshaw['nResult'], PPostshaw['nResult'], alternative='two-sided')

Ttest_indResult(statistic=0.3580838003027788, pvalue=0.7211284977971077)

Neponset:

In [20]:
stats.ttest_ind(PPrenep['nResult'], PPostnep['nResult'], alternative='two-sided')

Ttest_indResult(statistic=1.6924030282030538, pvalue=0.09327684838282967)

Charles:

In [21]:
stats.ttest_ind(PPrecharles['nResult'], PPostcharles['nResult'], alternative='two-sided')

Ttest_indResult(statistic=1.7425866048966383, pvalue=0.08283567330443037)

**Nitrate/Nitrite:**

These two show an even lower pval than the others, demonstrating more certain significance. 

In [22]:
nitrate_pre2013_df = df_copy.loc[(df_copy['StartDate'].dt.year<2013) & (df_copy['DWM_Name'] == 'Nitrate/Nitrite-N')]
nitrate_post2013_df = df_copy.loc[(df_copy['StartDate'].dt.year>=2013) & (df_copy['DWM_Name'] == 'Nitrate/Nitrite-N')]

stats.ttest_ind(nitrate_pre2013_df['nResult'], nitrate_post2013_df['nResult'], alternative='two-sided')



Ttest_indResult(statistic=3.402320058875278, pvalue=0.0007783526205137045)

In [96]:
print(nitrate_pre2013_df['nResult'].mean())
print(nitrate_post2013_df['nResult'].mean())

0.7447305389221558
0.33511904761904754


Once again, the t-tests for each watershed.

In [23]:
NPreSouth = nitrate_pre2013_df.loc[nitrate_pre2013_df['Watershed'] == 'South Coastal']
NPreshaw = nitrate_pre2013_df.loc[nitrate_pre2013_df['Watershed'] == 'Shawsheen']
NPrenep = nitrate_pre2013_df.loc[nitrate_pre2013_df['Watershed'] == 'Neponset']
NPrecharles = nitrate_pre2013_df.loc[nitrate_pre2013_df['Watershed'] == 'Charles']

NPostSouth = nitrate_post2013_df.loc[nitrate_post2013_df['Watershed'] == 'South Coastal']
NPostshaw = nitrate_post2013_df.loc[nitrate_post2013_df['Watershed'] == 'Shawsheen']
NPostnep = nitrate_post2013_df.loc[nitrate_post2013_df['Watershed'] == 'Neponset']
NPostcharles = nitrate_post2013_df.loc[nitrate_post2013_df['Watershed'] == 'Charles']

South Coastal:

In [24]:
stats.ttest_ind(NPreSouth['nResult'], NPostSouth['nResult'], alternative='two-sided')

Ttest_indResult(statistic=1.7899843202123182, pvalue=0.0814274868439968)

Shawsheen:

In [25]:
stats.ttest_ind(NPreshaw['nResult'], NPostshaw['nResult'], alternative='two-sided')

Ttest_indResult(statistic=nan, pvalue=nan)

Neponset:

In [26]:
stats.ttest_ind(NPrenep['nResult'], NPostnep['nResult'], alternative='two-sided')

Ttest_indResult(statistic=nan, pvalue=nan)

Charles:

In [27]:
stats.ttest_ind(NPrecharles['nResult'], NPostcharles['nResult'], alternative='two-sided')

Ttest_indResult(statistic=0.4527913909437512, pvalue=0.6512353126731127)

Nitrate/nitrite appears to have some missing data for the shawsheen and neponset watersheds. Otherwise, it seems like neither charles nor south coastal appear to show a significant difference, which is rather unusual compared to its original pvalue. Due to the missing data, however, we've decided to exclude nitrite/nitrate from our final analysis.

# Conclusions

Based on a significance level of 0.05, the following nutrients show significant differences for the watersheds they're listed under: 

South Coastal:
- Nitrogen
- Ammonia
- phosphorus

Shawsheen:
- Ammonia

Neponset: 
- Nitrogen

Charles: 
- None



It appears as though South Coastal is the only one significantly affected in general. This may have to do with South Coastal's location, southeast of the Boston Metropolitan Area. Surprisingly, Charles, which seems to pass through the heart of the Boston Metropolitan Area, doesn't have a significant difference in any nutrient level. The Shawsheen and Neponset watersheds only show a significant difference in the level of in one of 3 nutrients, indicating that they're is probably not much meaningful difference between pre- and post-2013 nutrient levels overall. 

In [84]:
AmmoniaAllSouthCoastal = df_copy.loc[(df_copy['DWM_Name'] == 'Ammonia-N') & (df_copy['Watershed'] == 'South Coastal')]
AmmoniaAllShaw = df_copy.loc[(df_copy['DWM_Name'] == 'Ammonia-N') & (df_copy['Watershed'] == 'Shawsheen')]
AmmoniaAllNep = df_copy.loc[(df_copy['DWM_Name'] == 'Ammonia-N') & (df_copy['Watershed'] == 'Neponset')]
AmmoniaAllCharles = df_copy.loc[(df_copy['DWM_Name'] == 'Ammonia-N') & (df_copy['Watershed'] == 'Charles')]

In [85]:
print('Nitrogen South Coastal Mean Distance', NitrogenAllSouthCoastal['closest_farm_distance_km'].mean())
print('Nitrogen Shawsheen Mean Distance', NitrogenAllShaw['closest_farm_distance_km'].mean())
print('Nitrogen Charles Mean Distance', NitrogenAllCharles['closest_farm_distance_km'].mean())
print('Nitrogen Neponset Mean Distance', NitrogenAllNep['closest_farm_distance_km'].mean())

print('Ammonia South Coastal Mean Distance', AmmoniaAllSouthCoastal['closest_farm_distance_km'].mean())
print('Ammonia Shawsheen Mean Distance', AmmoniaAllShaw['closest_farm_distance_km'].mean())
print('Ammonia Charles Mean Distance', AmmoniaAllCharles['closest_farm_distance_km'].mean())
print('Ammonia Neponset Mean Distance', AmmoniaAllNep['closest_farm_distance_km'].mean())



Nitrogen South Coastal Mean Distance 43.246770313169144
Nitrogen Shawsheen Mean Distance 26.6290762326017
Nitrogen Charles Mean Distance 18.496788935719223
Nitrogen Neponset Mean Distance 7.68816849433671
Ammonia South Coastal Mean Distance 37.21191830846172
Ammonia Shawsheen Mean Distance 26.718884498979097
Ammonia Charles Mean Distance 18.52659961761618
Ammonia Neponset Mean Distance 7.745954744667652


,Watershed,Latitude,Longitude,StartDate,StartTime,DWM_Name,DWM_Units,ResVal,nResult,dummy_variable,closest_farm_distance_km
131,Neponset,42.192183,-71.092968,2017-07-20,10:20:00 AM,Ammonia-N,mg/L,<0.04,-0.04,1.0,6.698472
135,Neponset,42.192183,-71.092968,2017-07-20,10:25:00 AM,Ammonia-N,mg/L,<0.04,-0.04,1.0,6.698472
155,Neponset,42.192183,-71.092968,2017-08-29,9:45:00 AM,Ammonia-N,mg/L,<0.04,-0.04,1.0,6.698472
159,Neponset,42.192183,-71.092968,2017-08-29,10:00:00 AM,Ammonia-N,mg/L,<0.04,-0.04,1.0,6.698472
351,Neponset,42.243430,-71.094378,2013-05-30,11:17:00 AM,Ammonia-N,mg/L,0.02,0.02,1.0,3.484433


Here I've determined that the Neponset watershed has the smallest mean distance to the urban farms in our database.